In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dl-kaggle-dataset/cleaned_train_x.csv
/kaggle/input/dl-kaggle-dataset/cleaned_val_x.csv
/kaggle/input/dl-kaggle-dataset/Train.py
/kaggle/input/dl-kaggle-dataset/models_zoo.py
/kaggle/input/dl-kaggle-dataset/cleaned_test_x.csv
/kaggle/input/dl-kaggle-dataset/utility.py
/kaggle/input/dl-kaggle-dataset/train_y.csv
/kaggle/input/dl-kaggle-dataset/train_x.csv
/kaggle/input/dl-kaggle-dataset/test_x.csv
/kaggle/input/dl-kaggle-dataset/glove.840B.300d.txt
/kaggle/input/dl-kaggle-dataset/DataPreprocessing.py
/kaggle/input/dl-kaggle-dataset/val_x.csv
/kaggle/input/dl-kaggle-dataset/val_y.csv
/kaggle/input/dl-kaggle-dataset/rnn_baseline.py
/kaggle/input/dl-kaggle-dataset/cleanwords.txt
/kaggle/input/dl-kaggle-dataset/DataLoader.py


In [2]:
import tensorflow as tf
# import module we'll need to import our custom module
from shutil import copyfile
# copy our file into the working directory (make sure it has .py suffix)
copyfile(src = "/kaggle/input/dl-kaggle-dataset/DataLoader.py", dst = "../working/DataLoader.py")
copyfile(src = "/kaggle/input/dl-kaggle-dataset/models_zoo.py", dst = "../working/models_zoo.py")
copyfile(src = "/kaggle/input/dl-kaggle-dataset/Train.py", dst = "../working/Train.py")
copyfile(src = "/kaggle/input/dl-kaggle-dataset/utility.py", dst = "../working/utility.py")
# import all our functions
from DataLoader import DataLoader
from Train import Trainer
from utility import worst_group_accuracy
from models_zoo import get_baseline, get_LSTM, get_LSTM_attn, get_GRU, get_gru_rnn_attention

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import torch
from keras.preprocessing.text import Tokenizer

glove_path = os.path.join(dirname, 'glove.840B.300d.txt')
embedding_path = [glove_path]
MAX_SEQUENCE_LENGTH = 350
MAX_FEATURES = 100000
EMBEDDING_DIM = 300
torch.manual_seed(0)
dataloader = DataLoader()
embedding_index = dataloader.load_embedding(embedding_path)

train_x = pd.read_csv(os.path.join(dirname, 'cleaned_train_x.csv'))
val_x = pd.read_csv(os.path.join(dirname, 'cleaned_val_x.csv'))
test_x = pd.read_csv(os.path.join(dirname, 'cleaned_test_x.csv'))

train_y = pd.read_csv(os.path.join(dirname, 'train_y.csv'))
val_y = pd.read_csv(os.path.join(dirname, 'val_y.csv'))
list_classes = ['y']
train_y, val_y = dataloader.load_dataset(train_x, train_y, val_x, val_y, test_x, list_classes)

tokenizer = Tokenizer(num_words = MAX_FEATURES)
train_x, test_x, val_x, word_index = dataloader.tokenize(tokenizer, MAX_SEQUENCE_LENGTH)
embedding_matrix = dataloader.create_embedding_matrix(word_index, EMBEDDING_DIM, embedding_index, MAX_FEATURES)

Total 2195884 word vectors
Shape of train_y : (269038, 1)
Shape of val_y : (45180, 1)
Shape of train_x tensor: (269038, 350)
Shape of test_data tensor: (133782, 350)
Shape of val_data tensor: (45180, 350)
Found 136016 unique tokens
Null word embeddings: 21362


In [4]:
print(train_x.shape)
print(train_y.shape)
print(val_x.shape)
print(val_y.shape)
print(test_x.shape)

(269038, 350)
(269038, 1)
(45180, 350)
(45180, 1)
(133782, 350)


In [6]:
MODEL_CHECKPOINT_FOLDER = "checkpoints/"
TEMPORARY_CHECKPOINTS_PATH = 'temporary_checkpoints/'
nb_words = min(MAX_FEATURES, len(word_index))
def get_model():
    return get_LSTM(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=1)

trainer = Trainer(model_stamp='get_LSTM', epoch_num=4, learning_rate=1e-3)
models,val_loss,total_auc,fold_predictions = trainer.train_folds(X=train_x, y=train_y, fold_count=4, batch_size=256, get_model_func=get_model)
print("Predicting val results...")
LSTM_val_predicts = []
for fold_id, model in enumerate(models):
    val_predicts = model.predict(val_x, batch_size=256, verbose=1)
    LSTM_val_predicts.append(val_predicts)
val_y_test = pd.read_csv(os.path.join(dirname, 'val_y.csv'))
folds_WGA = []
for fold in LSTM_val_predicts:
    pred_df = pd.DataFrame()
    pred_df['pred'] = [val[-1] for val in fold]
    pred_df = pred_df.reset_index()
    metric = worst_group_accuracy(pred_df, val_y_test)
    print(f'Fold_WGA: {metric}')
    print('-------------------------------------')
    folds_WGA.append(metric)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 350)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 350, 300)             3000000   ['input_1[0][0]']             
                                                          0                                       
                                                                                                  
 spatial_dropout1d (Spatial  (None, 350, 300)             0         ['embedding[0][0]']           
 Dropout1D)                                                                                       
                                                                                              

In [13]:
#######################################
### print out single model's performance
### store train set predictions 
### store val set predictions
### store test set predictions
#######################################
LSTM_score, LSTM_train_pred, LSTM_val_pred, LSTM_test_pred = save_single_model(models, val_x, test_x, fold_predictions, folds_WGA, 'LSTM')

523/523 [==============================] - 19s 36ms/step


In [46]:
LSTM_score

[0.7596852300242131,
 0.7766343825665859,
 0.7730024213075061,
 0.7723970944309927]

In [14]:
MODEL_CHECKPOINT_FOLDER = "checkpoints/"
TEMPORARY_CHECKPOINTS_PATH = 'temporary_checkpoints/'
nb_words = min(MAX_FEATURES, len(word_index))
def get_model():
    return get_GRU(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=1)

trainer = Trainer(model_stamp='gru', epoch_num=4, learning_rate=1e-3)
models,val_loss,total_auc,fold_predictions = trainer.train_folds(X=train_x, y=train_y, fold_count=4, batch_size=256, get_model_func=get_model)
print("Predicting val results...")
LSTM_val_predicts = []
for fold_id, model in enumerate(models):
    val_predicts = model.predict(val_x, batch_size=256, verbose=1)
    LSTM_val_predicts.append(val_predicts)
val_y_test = pd.read_csv(os.path.join(dirname, 'val_y.csv'))
folds_WGA = []
for fold in LSTM_val_predicts:
    pred_df = pd.DataFrame()
    pred_df['pred'] = [val[-1] for val in fold]
    pred_df = pred_df.reset_index()
    metric = worst_group_accuracy(pred_df, val_y_test)
    print(f'Fold_WGA: {metric}')
    folds_WGA.append(metric)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 350)]                0         []                            
                                                                                                  
 embedding_4 (Embedding)     (None, 350, 300)             3000000   ['input_5[0][0]']             
                                                          0                                       
                                                                                                  
 spatial_dropout1d_4 (Spati  (None, 350, 300)             0         ['embedding_4[0][0]']         
 alDropout1D)                                                                                     
                                                                                            

In [15]:
GRU_score, GRU_train_pred, GRU_val_pred, GRU_test_pred = save_single_model(models, val_x, test_x, fold_predictions, folds_WGA, 'GRU')

523/523 [==============================] - 19s 37ms/step


In [45]:
GRU_score

[0.761501210653753, 0.7663438256658596, 0.7639225181598063, 0.763317191283293]

In [16]:
MODEL_CHECKPOINT_FOLDER = "checkpoints/"
TEMPORARY_CHECKPOINTS_PATH = 'temporary_checkpoints/'
nb_words = min(MAX_FEATURES, len(word_index))
def get_model():
    return  get_gru_rnn_attention(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=1)

trainer = Trainer(model_stamp='gru_attn_rnn', epoch_num=4, learning_rate=1e-3)
models,val_loss,total_auc,fold_predictions = trainer.train_folds(X=train_x, y=train_y, fold_count=4, batch_size=256, get_model_func=get_model)
print("Predicting val results...")
LSTM_val_predicts = []
for fold_id, model in enumerate(models):
    val_predicts = model.predict(val_x, batch_size=256, verbose=1)
    LSTM_val_predicts.append(val_predicts)
val_y_test = pd.read_csv(os.path.join(dirname, 'val_y.csv'))
folds_WGA = []
for fold in LSTM_val_predicts:
    pred_df = pd.DataFrame()
    pred_df['pred'] = [val[-1] for val in fold]
    pred_df = pred_df.reset_index()
    metric = worst_group_accuracy(pred_df, val_y_test)
    print(f'Fold_WGA: {metric}')
    folds_WGA.append(metric)

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 350)]                0         []                            
                                                                                                  
 embedding_8 (Embedding)     (None, 350, 300)             3000000   ['input_9[0][0]']             
                                                          0                                       
                                                                                                  
 spatial_dropout1d_8 (Spati  (None, 350, 300)             0         ['embedding_8[0][0]']         
 alDropout1D)                                                                                     
                                                                                            

In [17]:
GRU_attn_score, GRU_attn_train_pred, GRU_attn_val_pred, GRU_attn_test_pred = save_single_model(models, val_x, test_x, fold_predictions, folds_WGA, 'GRU_attn')

523/523 [==============================] - 20s 38ms/step


In [44]:
GRU_attn_score

[0.7645278450363197,
 0.7736077481840193,
 0.7584745762711864,
 0.7760290556900726]

In [26]:
train_x = pd.DataFrame()
train_x['LSTM'] = [val[0] for val in LSTM_train_pred]
train_x['GRU'] = [val[0] for val in GRU_train_pred]
train_x['GRU_attn'] = [val[0] for val in GRU_attn_train_pred]
train_x

,LSTM,GRU,GRU_attn
0,0.469449,0.350979,0.369549
1,0.202535,0.290206,0.408400
2,0.622117,0.692984,0.654164
3,0.438028,0.553940,0.469837
4,0.640644,0.622073,0.664755
...,...,...,...
269033,0.239672,0.218813,0.231567
269034,0.255629,0.316450,0.217420
269035,0.014334,0.021539,0.006388
269036,0.107205,0.176530,0.095793


In [29]:
val_x = pd.DataFrame()
val_x['LSTM'] = [val[0] for val in LSTM_val_pred]
val_x['GRU'] = [val[0] for val in GRU_val_pred]
val_x['GRU_attn'] = [val[0] for val in GRU_attn_val_pred]
val_x

,LSTM,GRU,GRU_attn
0,0.855428,0.903790,0.907688
1,0.796110,0.810574,0.866206
2,0.845888,0.898856,0.888335
3,0.374033,0.503030,0.471473
4,0.669301,0.596998,0.652607
...,...,...,...
45175,0.098179,0.069736,0.035183
45176,0.080056,0.050775,0.097586
45177,0.144540,0.169164,0.181009
45178,0.101906,0.086000,0.054459


In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
model = SVC()
model.fit(train_x, train_y)
predictions = model.predict(val_x)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


NameError: name 'train_x' is not defined

In [42]:
stacking_val_pred = pd.DataFrame()
stacking_val_pred['pred'] = [val[0] for val in predictions]
stacking_val_pred = stacking_val_pred.reset_index()
stacking_val_pred

,index,pred
0,0,0.946137
1,1,0.872145
2,2,0.935203
3,3,0.476243
4,4,0.673089
...,...,...
45175,45175,0.064982
45176,45176,0.067351
45177,45177,0.165959
45178,45178,0.078862


In [43]:
stacking_metric = worst_group_accuracy(stacking_val_pred, val_y_test)
print(f'stacking WGA: {stacking_metric}')

male_1: 0.9003147953830011
female_1: 0.9088439993209981
LGBTQ_1: 0.8071379547014413
christian_1: 0.9331868131868132
muslim_1: 0.8298578199052132
other_religions_1: 0.8640973630831643
black_1: 0.7772397094430993
white_1: 0.7907220090687129
stacking WGA: 0.7772397094430993


In [10]:
MODEL_CHECKPOINT_FOLDER = "checkpoints/"
TEMPORARY_CHECKPOINTS_PATH = 'temporary_checkpoints/'
nb_words = min(MAX_FEATURES, len(word_index))
def get_model():
    return  get_textCNN(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=1)

trainer = Trainer(model_stamp='textCNN', epoch_num=4, learning_rate=1e-3)
models,val_loss,total_auc,fold_predictions = trainer.train_folds(X=train_x, y=train_y, fold_count=4, batch_size=256, get_model_func=get_model)
print("Predicting val results...")
LSTM_val_predicts = []
for fold_id, model in enumerate(models):
    val_predicts = model.predict(val_x, batch_size=256, verbose=1)
    LSTM_val_predicts.append(val_predicts)
val_y_test = pd.read_csv(os.path.join(dirname, 'val_y.csv'))
folds_WGA = []
for fold in LSTM_val_predicts:
    pred_df = pd.DataFrame()
    pred_df['pred'] = [val[-1] for val in fold]
    pred_df = pred_df.reset_index()
    metric = worst_group_accuracy(pred_df, val_y_test)
    print(f'Fold_WGA: {metric}')
    folds_WGA.append(metric)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 350)]                0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, 350, 300)             3000000   ['input_3[0][0]']             
                                                          0                                       
                                                                                                  
 spatial_dropout1d_2 (Spati  (None, 350, 300)             0         ['embedding_2[0][0]']         
 alDropout1D)                                                                                     
                                                                                              

KeyboardInterrupt: 